# Pipeline Completo: Forecast Promtur - Tráfico Orgánico

**Proyecto:** Predicción de métricas de tráfico orgánico 2026
**Última actualización:** Noviembre 2025

---

## Contenido:

1. **Configuración del entorno** (Local/Colab)
2. **Análisis exploratorio** de datos históricos
3. **Preparación y limpieza** de datos
4. **Modelos de forecasting** con Prophet
5. **Visualización y reportes** finales
6. **Descarga de resultados**

---

## Instrucciones:

### En Google Colab:
1. Ejecuta todas las celdas (Runtime → Run all)
2. Sube tu CSV de GA4 cuando se te pida
3. Los resultados se descargarán automáticamente al final

### En entorno local:
1. Asegúrate de tener el CSV en `data/raw/ga4_promtur_organic_2025.csv`
2. Ejecuta todas las celdas secuencialmente
3. Los resultados se guardarán en las carpetas correspondientes

---
# SECCIÓN 0: CONFIGURACIÓN DEL ENTORNO
---

In [ ]:
# ===================================================================
# DETECCIÓN AUTOMÁTICA DE ENTORNO (Local vs Colab)
# ===================================================================

import sys
from pathlib import Path

# Detectar si estamos en Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔵 EJECUTANDO EN GOOGLE COLAB")
    print("=" * 70)
    
    # 1. Instalar dependencias
    print("\n📦 Instalando librerías necesarias...")
    !pip install -q prophet openpyxl
    print("✅ Librerías instaladas")
    
    # 2. Crear estructura de carpetas
    print("\n📁 Creando estructura de carpetas...")
    !mkdir -p data/raw data/processed data/forecasts
    !mkdir -p results/figures/exploratory results/figures/forecasts results/figures/final
    !mkdir -p results/reports
    print("✅ Carpetas creadas")
    
    # 3. Upload de archivo CSV
    from google.colab import files
    import shutil
    
    print("\n" + "=" * 70)
    print("📊 SUBIR ARCHIVO DE DATOS")
    print("=" * 70)
    print("\nPor favor, sube tu archivo CSV de GA4 con los siguientes datos:")
    print("  - Year")
    print("  - Month number")
    print("  - Session Default Channel Group Custom (Recovery)")
    print("  - Sessions - GA4")
    print("  - Bounces")
    print("  - Total session duration - GA4")
    print("  - Views - GA4")
    print("\n👇 Haz clic en 'Choose Files' para subir tu CSV:\n")
    
    uploaded = files.upload()
    
    # Mover CSV a data/raw/
    for filename in uploaded.keys():
        # Renombrar al nombre esperado
        shutil.move(filename, 'data/raw/ga4_promtur_organic_2025.csv')
        print(f"\n✅ Archivo guardado como: data/raw/ga4_promtur_organic_2025.csv")
    
    # Definir rutas para Colab
    DATA_RAW = Path('data/raw')
    DATA_PROCESSED = Path('data/processed')
    DATA_FORECASTS = Path('data/forecasts')
    RESULTS_FIGURES_EXPLORATORY = Path('results/figures/exploratory')
    RESULTS_FIGURES_FORECASTS = Path('results/figures/forecasts')
    RESULTS_FIGURES_FINAL = Path('results/figures/final')
    RESULTS_REPORTS = Path('results/reports')
    
    print("\n" + "=" * 70)
    print("✅ CONFIGURACIÓN DE COLAB COMPLETADA")
    print("=" * 70)
    print("\n🚀 Procede a ejecutar el resto del notebook\n")
    
else:
    print("🟢 EJECUTANDO EN ENTORNO LOCAL")
    print("=" * 70)
    
    # Definir rutas para entorno local
    DATA_RAW = Path('../data/raw')
    DATA_PROCESSED = Path('../data/processed')
    DATA_FORECASTS = Path('../data/forecasts')
    RESULTS_FIGURES_EXPLORATORY = Path('../results/figures/exploratory')
    RESULTS_FIGURES_FORECASTS = Path('../results/figures/forecasts')
    RESULTS_FIGURES_FINAL = Path('../results/figures/final')
    RESULTS_REPORTS = Path('../results/reports')
    
    # Crear carpetas si no existen
    for folder in [DATA_RAW, DATA_PROCESSED, DATA_FORECASTS, 
                   RESULTS_FIGURES_EXPLORATORY, RESULTS_FIGURES_FORECASTS, 
                   RESULTS_FIGURES_FINAL, RESULTS_REPORTS]:
        folder.mkdir(parents=True, exist_ok=True)
    
    print("✅ Entorno local configurado")
    print("\n🚀 Procede a ejecutar el resto del notebook\n")

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from datetime import timedelta
import warnings

# Ignorar warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Semilla para reproducibilidad
np.random.seed(42)

print("✅ Librerías importadas correctamente")

In [ ]:
# Función auxiliar para formatear duración
def segundos_a_hhmm_ss(segundos):
    """
    Convierte segundos a formato HH:MM:SS
    """
    if pd.isna(segundos) or segundos < 0:
        return "00:00:00"
    
    horas = int(segundos // 3600)
    minutos = int((segundos % 3600) // 60)
    segs = int(segundos % 60)
    
    return f"{horas:02d}:{minutos:02d}:{segs:02d}"

print("✅ Funciones auxiliares creadas")

---
# SECCIÓN 1: ANÁLISIS EXPLORATORIO
---

## 1.1 Carga y exploración inicial de datos

In [ ]:
# Cargar dataset
csv_file = DATA_RAW / 'ga4_promtur_organic_2025.csv'

if csv_file.exists():
    df_raw = pd.read_csv(csv_file)
    print(f"✅ Dataset cargado exitosamente")
    print(f"📊 Dimensiones: {df_raw.shape[0]} filas x {df_raw.shape[1]} columnas\n")
else:
    print(f"❌ Error: No se encontró el archivo {csv_file}")

In [ ]:
# Configuración de nombres de columnas originales
year_col = 'Year'
month_col = 'Month number'
canal_col = 'Session Default Channel Group Custom (Recovery)'
sessions_col = 'Sessions - GA4'
bounces_col = 'Bounces'
duration_col = 'Total session duration - GA4'
views_col = 'Views - GA4'

print("✅ Variables de columnas configuradas")

In [ ]:
# Primeras filas
print("📋 Primeras 10 filas del dataset:\n")
display(df_raw.head(10))

In [ ]:
# Información del dataset
print("ℹ️ Información del dataset:\n")
df_raw.info()

In [ ]:
# Verificar canales y rango temporal
print("🎯 Canales únicos encontrados:\n")
print(df_raw[canal_col].value_counts().sort_index())

print(f"\n📅 Rango temporal:")
print(f"   Año(s): {sorted(df_raw[year_col].unique())}")
print(f"   Meses: {sorted(df_raw[month_col].unique())}")
print(f"   Total de meses únicos: {df_raw[[year_col, month_col]].drop_duplicates().shape[0]}")

## 1.2 Calidad de datos

In [ ]:
# Verificar valores faltantes
missing = df_raw.isnull().sum()
if missing.sum() > 0:
    print("⚠️ Valores faltantes encontrados:\n")
    display(missing[missing > 0])
else:
    print("✅ No hay valores faltantes")

# Verificar duplicados
duplicados = df_raw.duplicated().sum()
print(f"\n🔍 Registros duplicados: {duplicados}")
if duplicados == 0:
    print("✅ No hay registros duplicados")

## 1.3 Visualizaciones exploratorias

In [ ]:
# Crear columna de fecha
df_raw['fecha'] = pd.to_datetime(
    df_raw[year_col].astype(str) + '-' + df_raw[month_col].astype(str) + '-01'
)

# Gráfico: Evolución de sesiones por canal
fig, ax = plt.subplots(figsize=(14, 6))

for canal in sorted(df_raw[canal_col].unique()):
    data_canal = df_raw[df_raw[canal_col] == canal].sort_values('fecha')
    ax.plot(data_canal['fecha'], data_canal[sessions_col], 
            marker='o', label=canal, linewidth=2, markersize=6)

ax.set_xlabel('Mes', fontsize=12)
ax.set_ylabel('Sesiones', fontsize=12)
ax.set_title('Evolución de Sesiones por Canal (2025)', fontsize=14, fontweight='bold')
ax.legend(title='Canal', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(RESULTS_FIGURES_EXPLORATORY / 'sessions_by_channel.png', dpi=300, bbox_inches='tight')
plt.show()

print("💾 Gráfico guardado")

---
# SECCIÓN 2: PREPARACIÓN Y LIMPIEZA DE DATOS
---

## 2.1 Transformación a snake_case

In [ ]:
# Mapeo de columnas a snake_case
column_mapping = {
    'Year': 'year',
    'Month number': 'month',
    'Session Default Channel Group Custom (Recovery)': 'channel',
    'Sessions - GA4': 'sessions',
    'Bounces': 'bounces',
    'Total session duration - GA4': 'total_session_duration',
    'Views - GA4': 'views'
}

df_clean = df_raw.rename(columns=column_mapping)

print("✅ Columnas renombradas a snake_case")
print(f"\nColumnas del dataset limpio: {list(df_clean.columns)}")

## 2.2 Cálculo de métricas derivadas

In [ ]:
# Calcular métricas derivadas
df_clean['bounce_rate'] = (df_clean['bounces'] / df_clean['sessions']) * 100
df_clean['views_per_session'] = df_clean['views'] / df_clean['sessions']
df_clean['avg_session_duration'] = df_clean['total_session_duration'] / df_clean['sessions']

# Crear columna de fecha
df_clean['ds'] = pd.to_datetime(
    df_clean['year'].astype(str) + '-' + df_clean['month'].astype(str) + '-01'
)

print("✅ Métricas derivadas calculadas:")
print("   - bounce_rate (%)")
print("   - views_per_session")
print("   - avg_session_duration (segundos)")

# Mostrar muestra
print("\n📊 Muestra de datos procesados:\n")
display(df_clean[['year', 'month', 'channel', 'sessions', 'bounce_rate', 
                   'views_per_session', 'avg_session_duration']].head(10))

## 2.3 Filtrado de canales (OPCIONAL)

In [ ]:
# Configuración: usar todos los canales o filtrar
usar_todos_los_canales = True

if usar_todos_los_canales:
    df_final = df_clean.copy()
    print(f"✅ Usando TODOS los canales ({df_final['channel'].nunique()} canales)")
    print(f"\nCanales incluidos:")
    for canal in sorted(df_final['channel'].unique()):
        print(f"   - {canal}")
else:
    # Puedes personalizar esta lista
    canales_incluir = [    
    'Organic Search',
    #'Direct',
    #'Referral',
    #'Organic Social',
    'AI Traffic',
    #'Email'
    #'Organic Video',
    #'QR Code',
    #'Organic Shopping',
    # Agrega o quita canales según necesites
    ]
    df_final = df_clean[df_clean['channel'].isin(canales_incluir)].copy()
    print(f"✅ Filtrado aplicado: {len(canales_incluir)} canales seleccionados")

## 2.4 Guardado de dataset limpio

In [ ]:
# Seleccionar columnas finales
columnas_finales = [
    'year', 'month', 'channel', 'sessions', 'bounces', 
    'total_session_duration', 'views', 'bounce_rate', 
    'views_per_session', 'avg_session_duration', 'ds'
]

df_output = df_final[columnas_finales].copy()
df_output = df_output.sort_values(['year', 'month', 'channel']).reset_index(drop=True)

# Guardar
output_file = DATA_PROCESSED / 'dataset_clean.csv'
df_output.to_csv(output_file, index=False)

print(f"✅ Dataset limpio guardado en: {output_file}")
print(f"📊 Dimensiones: {df_output.shape[0]} filas x {df_output.shape[1]} columnas")

---
# SECCIÓN 3: MODELOS DE FORECASTING
---

## 3.1 Configuración de forecasting

In [ ]:
# Configuración
metricas_forecast = ['sessions', 'bounce_rate', 'views_per_session', 'avg_session_duration']
canales = sorted(df_output['channel'].unique())
periodos_forecast = 12  # 12 meses de 2026

print(f"📊 Configuración de forecasting:")
print(f"\n   Métricas: {len(metricas_forecast)}")
for m in metricas_forecast:
    print(f"      - {m}")
print(f"\n   Canales: {len(canales)}")
for c in canales:
    print(f"      - {c}")
print(f"\n   Horizonte: {periodos_forecast} meses (2026)")
print(f"\n   Total de modelos: {len(metricas_forecast)} × {len(canales)} = {len(metricas_forecast) * len(canales)}")

## 3.2 Función de forecasting

In [ ]:
def crear_forecast_prophet(df_canal, metrica, periodos=12):
    """
    Crea forecast con Prophet
    """
    # Preparar datos
    df_prophet = df_canal[['ds', metrica]].rename(columns={metrica: 'y'})
    
    # Modelo
    model = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        interval_width=0.95
    )
    
    model.fit(df_prophet)
    
    # Predicciones
    future = model.make_future_dataframe(periods=periodos, freq='MS')
    forecast = model.predict(future)
    
    # Métricas de evaluación
    forecast_hist = forecast[forecast['ds'].isin(df_prophet['ds'])]
    valores_reales = df_prophet['y'].values
    valores_pred = forecast_hist['yhat'].values
    
    mae = np.mean(np.abs(valores_reales - valores_pred))
    rmse = np.sqrt(np.mean((valores_reales - valores_pred)**2))
    mape = np.mean(np.abs((valores_reales - valores_pred) / valores_reales)) * 100
    
    return model, forecast, {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

print("✅ Función de forecasting creada")

## 3.3 Entrenamiento de modelos

In [ ]:
# Entrenar modelos
resultados_forecasts = {}
metricas_evaluacion = {}

print("🚀 Entrenando modelos...\n")
print("="*70)

total = len(canales) * len(metricas_forecast)
actual = 0

for canal in canales:
    print(f"\n📊 Canal: {canal}")
    print("-" * 70)
    
    df_canal = df_output[df_output['channel'] == canal].sort_values('ds')
    resultados_forecasts[canal] = {}
    metricas_evaluacion[canal] = {}
    
    for metrica in metricas_forecast:
        actual += 1
        print(f"   [{actual}/{total}] {metrica}...", end=" ")
        
        model, forecast, metrics = crear_forecast_prophet(df_canal, metrica, periodos_forecast)
        
        resultados_forecasts[canal][metrica] = forecast
        metricas_evaluacion[canal][metrica] = metrics
        
        print(f"MAPE: {metrics['MAPE']:.2f}%")

print("\n" + "="*70)
print("✅ Todos los modelos entrenados")

## 3.4 Exportación de predicciones

In [ ]:
# Consolidar predicciones 2026
predicciones_2026 = []

for canal in canales:
    for metrica in metricas_forecast:
        forecast = resultados_forecasts[canal][metrica]
        forecast_2026 = forecast[forecast['ds'].dt.year == 2026][['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
        
        forecast_2026['channel'] = canal
        forecast_2026['metric'] = metrica
        forecast_2026['year'] = 2026
        forecast_2026['month'] = forecast_2026['ds'].dt.month
        
        forecast_2026 = forecast_2026.rename(columns={
            'yhat': 'predicted_value',
            'yhat_lower': 'lower_bound',
            'yhat_upper': 'upper_bound'
        })
        
        predicciones_2026.append(forecast_2026)

df_predicciones = pd.concat(predicciones_2026, ignore_index=True)
df_predicciones = df_predicciones[[
    'year', 'month', 'channel', 'metric', 
    'predicted_value', 'lower_bound', 'upper_bound', 'ds'
]].sort_values(['channel', 'metric', 'year', 'month']).reset_index(drop=True)

# Guardar
forecast_file = DATA_FORECASTS / 'forecasts_2026_all_channels.csv'
df_predicciones.to_csv(forecast_file, index=False)

print(f"✅ Predicciones guardadas en: {forecast_file}")
print(f"📊 Total de predicciones: {len(df_predicciones)}")

---
# SECCIÓN 4: VISUALIZACIÓN Y REPORTES
---

## 4.1 Ajuste de bounce_rate (0-100%)

In [ ]:
# Limitar bounce_rate a 0-100%
mask_bounce = df_predicciones['metric'] == 'bounce_rate'

print("🔧 Aplicando limitación de bounce_rate a 0-100%...")

df_predicciones.loc[mask_bounce, 'predicted_value'] = df_predicciones.loc[mask_bounce, 'predicted_value'].clip(0, 100)
df_predicciones.loc[mask_bounce, 'lower_bound'] = df_predicciones.loc[mask_bounce, 'lower_bound'].clip(0, 100)
df_predicciones.loc[mask_bounce, 'upper_bound'] = df_predicciones.loc[mask_bounce, 'upper_bound'].clip(0, 100)

print("✅ Bounce_rate limitado correctamente")

## 4.2 Análisis de confiabilidad

In [ ]:
# Identificar canales poco confiables
canales_confiabilidad = []

for canal in canales:
    df_canal_pred = df_predicciones[df_predicciones['channel'] == canal]
    df_canal_hist = df_output[df_output['channel'] == canal]
    
    volumen = df_canal_hist['sessions'].mean()
    
    metricas_positivas = ['sessions', 'views_per_session', 'avg_session_duration']
    negativos = df_canal_pred[
        df_canal_pred['metric'].isin(metricas_positivas) & 
        (df_canal_pred['predicted_value'] < 0)
    ].shape[0]
    
    problemas = []
    if volumen < 100:
        problemas.append('Bajo volumen')
    if negativos > 0:
        problemas.append('Valores negativos')
    
    confiabilidad = 'BAJA' if problemas else ('MEDIA' if volumen < 1000 else 'ALTA')
    
    canales_confiabilidad.append({
        'canal': canal,
        'volumen_promedio': volumen,
        'confiabilidad': confiabilidad,
        'problemas': ', '.join(problemas) if problemas else 'Ninguno'
    })

df_confiabilidad = pd.DataFrame(canales_confiabilidad).sort_values('volumen_promedio', ascending=False)

print("📊 Análisis de confiabilidad:\n")
display(df_confiabilidad)

# Guardar
df_confiabilidad.to_csv(RESULTS_REPORTS / 'canales_confiabilidad.csv', index=False)
print(f"\n💾 Análisis guardado")

## 4.3 Tablas resumen por canal

In [ ]:
# Crear tablas resumen (métricas × meses) con duración en HH:MM:SS
def crear_tabla_resumen(canal):
    df_canal = df_predicciones[df_predicciones['channel'] == canal].copy()
    df_canal['mes'] = df_canal['ds'].dt.strftime('%b-%y')
    
    tabla = df_canal.pivot(index='metric', columns='mes', values='predicted_value')
    
    meses_orden = df_canal.sort_values('ds')['mes'].unique()
    tabla = tabla[meses_orden]
    tabla['Promedio'] = tabla.mean(axis=1)
    
    # Crear fila adicional con duración en formato HH:MM:SS
    if 'avg_session_duration' in tabla.index:
        duracion_seg = tabla.loc['avg_session_duration'].copy()
        duracion_formateada = duracion_seg.apply(segundos_a_hhmm_ss)
        
        # Agregar fila formateada justo después de la fila en segundos
        tabla.loc['avg_session_duration_formatted'] = duracion_formateada
    
    nombres = {
        'sessions': 'Sesiones',
        'bounce_rate': 'Bounce Rate (%)',
        'views_per_session': 'Vistas/Sesión',
        'avg_session_duration': 'Duración Avg (seg)',
        'avg_session_duration_formatted': 'Duración Avg (HH:MM:SS)'
    }
    tabla = tabla.rename(index=nombres)
    
    return tabla.round(2)

# Generar y mostrar tablas
print("📊 TABLAS RESUMEN POR CANAL\n")
print("="*80)

tablas = {}
for canal in canales:
    print(f"\n📈 {canal}")
    print("-"*80)
    
    conf = df_confiabilidad[df_confiabilidad['canal'] == canal]['confiabilidad'].values[0]
    if conf == 'BAJA':
        print("⚠️ ADVERTENCIA: Confiabilidad BAJA\n")
    
    tabla = crear_tabla_resumen(canal)
    tablas[canal] = tabla
    display(tabla)

print("\n" + "="*80)

In [ ]:
# Exportar tablas a Excel con formato HH:MM:SS
excel_file = RESULTS_REPORTS / 'tablas_resumen_2026.xlsx'

with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    for canal, tabla in tablas.items():
        sheet_name = canal[:31]
        tabla.to_excel(writer, sheet_name=sheet_name)

print(f"💾 Tablas exportadas a: {excel_file}")
print(f"\n✅ Nota: Las tablas incluyen duración en segundos Y en formato HH:MM:SS")

## 4.4 Gráficos comparativos (TODAS las métricas)

In [ ]:
# Generar gráficos para TODAS las métricas de TODOS los canales
print("📊 Generando gráficos comparativos para todas las métricas...\n")

metricas_titulos = {
    'sessions': 'Sesiones',
    'bounce_rate': 'Bounce Rate (%)',
    'views_per_session': 'Vistas por Sesión',
    'avg_session_duration': 'Duración Promedio (segundos)'
}

graficos_generados = 0

for canal in canales:
    print(f"\n📈 Canal: {canal}")
    print("-" * 70)
    
    for metrica, titulo in metricas_titulos.items():
        print(f"   Gráfico: {titulo}")
        
        fig, ax = plt.subplots(figsize=(14, 6))
        
        # Histórico
        df_hist = df_output[df_output['channel'] == canal].sort_values('ds')
        ax.plot(df_hist['ds'], df_hist[metrica], 
                'o-', color='black', label='Histórico 2025', linewidth=2.5, markersize=7)
        
        # Predicción
        df_pred = df_predicciones[(df_predicciones['channel'] == canal) & 
                                  (df_predicciones['metric'] == metrica)].sort_values('ds')
        
        conf = df_confiabilidad[df_confiabilidad['canal'] == canal]['confiabilidad'].values[0]
        color = '#0072B2' if conf in ['ALTA', 'MEDIA'] else '#D55E00'
        linestyle = '-' if conf in ['ALTA', 'MEDIA'] else '--'
        
        ax.plot(df_pred['ds'], df_pred['predicted_value'], 
                'o-', color=color, label=f'Predicción 2026 ({conf})', 
                linewidth=2.5, markersize=7, linestyle=linestyle)
        
        ax.fill_between(df_pred['ds'], df_pred['lower_bound'], df_pred['upper_bound'],
                        alpha=0.2, color=color, label='Intervalo 95%')
        
        # Línea separadora
        fecha_sep = df_hist['ds'].max() + pd.DateOffset(days=15)
        ax.axvline(x=fecha_sep, color='gray', linestyle=':', linewidth=1.5, alpha=0.7)
        
        # Advertencia si baja confiabilidad
        if conf == 'BAJA':
            problemas = df_confiabilidad[df_confiabilidad['canal'] == canal]['problemas'].values[0]
            ax.text(0.5, 0.95, f'⚠️ ADVERTENCIA: {problemas}', 
                    transform=ax.transAxes, fontsize=10, color='red',
                    ha='center', va='top', bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
        
        ax.set_title(f'{titulo} - {canal}\nHistórico 2025 vs Predicción 2026', 
                     fontsize=14, fontweight='bold')
        ax.set_xlabel('Fecha', fontsize=12)
        ax.set_ylabel(titulo, fontsize=12)
        ax.legend(loc='best', fontsize=9)
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        filename = f"comp_{metrica}_{canal.replace(' ', '_').lower()}.png"
        plt.savefig(RESULTS_FIGURES_FINAL / filename, dpi=300, bbox_inches='tight')
        plt.show()
        
        graficos_generados += 1

print(f"\n" + "="*70)
print(f"✅ {graficos_generados} gráficos generados y guardados")
print(f"📁 Ubicación: {RESULTS_FIGURES_FINAL}")

## 4.5 Resumen ejecutivo

In [ ]:
# Resumen ejecutivo final
print("="*80)
print("🎯 RESUMEN EJECUTIVO - PREDICCIONES 2026")
print("="*80)

print("\n📊 PREDICCIONES POR CANAL:\n")

for canal in sorted(canales):
    conf = df_confiabilidad[df_confiabilidad['canal'] == canal]['confiabilidad'].values[0]
    icono = '✅' if conf == 'ALTA' else ('⚠️' if conf == 'MEDIA' else '🚨')
    
    print(f"\n{icono} {canal} (Confiabilidad: {conf})")
    
    df_canal = df_predicciones[df_predicciones['channel'] == canal]
    
    sessions = df_canal[df_canal['metric'] == 'sessions']['predicted_value'].sum()
    bounce = df_canal[df_canal['metric'] == 'bounce_rate']['predicted_value'].mean()
    vps = df_canal[df_canal['metric'] == 'views_per_session']['predicted_value'].mean()
    dur = df_canal[df_canal['metric'] == 'avg_session_duration']['predicted_value'].mean()
    
    print(f"   Sesiones totales 2026: {sessions:,.0f}")
    print(f"   Bounce Rate promedio: {bounce:.1f}%")
    print(f"   Vistas/Sesión: {vps:.2f}")
    print(f"   Duración promedio: {segundos_a_hhmm_ss(dur)} ({dur:.0f} seg)")

print("\n" + "="*80)
print("\n✅ ANÁLISIS COMPLETADO")
print("="*80)

---
# SECCIÓN 5: DESCARGA DE RESULTADOS (Solo Colab)
---

In [ ]:
if IN_COLAB:
    from google.colab import files
    import zipfile
    import os
    
    print("📦 Preparando archivos para descarga...\n")
    
    # Crear ZIP con todos los resultados
    zip_filename = 'resultados_forecast_promtur.zip'
    
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        # Agregar CSVs
        for file in ['data/processed/dataset_clean.csv', 
                     'data/forecasts/forecasts_2026_all_channels.csv',
                     'results/reports/canales_confiabilidad.csv']:
            if os.path.exists(file):
                zipf.write(file)
        
        # Agregar Excel
        if os.path.exists('results/reports/tablas_resumen_2026.xlsx'):
            zipf.write('results/reports/tablas_resumen_2026.xlsx')
        
        # Agregar gráficos
        for root, dirs, files_list in os.walk('results/figures'):
            for file in files_list:
                if file.endswith('.png'):
                    filepath = os.path.join(root, file)
                    zipf.write(filepath)
    
    print(f"✅ Archivo ZIP creado: {zip_filename}")
    print(f"📊 Total de gráficos incluidos: {graficos_generados}")
    print("\n📥 Descargando resultados...\n")
    
    files.download(zip_filename)
    
    print("\n" + "="*80)
    print("✅ DESCARGA COMPLETADA")
    print("="*80)
    print("\nEl archivo ZIP contiene:")
    print("  - Dataset limpio (CSV)")
    print("  - Predicciones 2026 (CSV)")
    print("  - Tablas resumen por canal (Excel con HH:MM:SS)")
    print("  - Análisis de confiabilidad (CSV)")
    print(f"  - {graficos_generados} gráficos comparativos (PNG)")
else:
    print("🟢 Entorno local: Todos los archivos están guardados en sus carpetas correspondientes")
    print(f"\n📊 Total de gráficos generados: {graficos_generados}")

---

## ¡ANÁLISIS COMPLETADO!

### Archivos generados:

**Datos:**
- `data/processed/dataset_clean.csv` - Dataset limpio
- `data/forecasts/forecasts_2026_all_channels.csv` - Predicciones 2026

**Reportes:**
- `results/reports/tablas_resumen_2026.xlsx` - Tablas por canal con duración en HH:MM:SS
- `results/reports/canales_confiabilidad.csv` - Análisis de confiabilidad

**Visualizaciones:**
- `results/figures/exploratory/` - Gráficos exploratorios
- `results/figures/final/` - Gráficos comparativos de TODAS las métricas

---

### ⚠️ Consideraciones importantes:

1. **Bounce Rate**: Limitado automáticamente a rango 0-100%
2. **Canales con baja confiabilidad**: Señalizados en gráficos y análisis
3. **Intervalos de confianza**: Reflejan incertidumbre del modelo
4. **Duración de sesión**: Disponible en segundos y formato HH:MM:SS

### Recomendaciones:

- Usar predicciones de canales con confiabilidad ALTA/MEDIA
- Considerar intervalos de confianza en planificación
- Actualizar modelos con datos reales de 2026
- Incorporar datos de 2024 cuando estén disponibles

---

**Fecha:** Noviembre 2025